In [34]:
import pandas as pd
url_discentes = "http://dados.ufrn.br/dataset/554c2d41-cfce-4278-93c6-eb9aa49c5d16/resource/a55aef81-e094-4267-8643-f283524e3dd7/download/discentes-2019.csv"
discentes = pd.read_csv(url_discentes, sep=';')
dados_discentes = discentes[['sexo', 'raca', 'tipo_discente', 'status', 'sigla_nivel_ensino', 'nome_curso']]
dados_discentes = dados_discentes[(dados_discentes['status'] == 'ATIVO') & (dados_discentes['sigla_nivel_ensino'] == 'G')]

url_cursos = "http://dados.ufrn.br/dataset/02526b96-cf40-4507-90b0-3afe5ddd53e7/resource/a10bc434-9a2d-491a-ae8c-41cf643c35bc/download/cursos-de-graduacao.csv"
cursos = pd.read_csv(url_cursos, sep=';')
dados_cursos = cursos[['nome', 'nivel_ensino', 'area_conhecimento']]
dados_cursos = dados_cursos[(dados_cursos['nivel_ensino'] == 'GRADUAÇÃO')]

,nome,nivel_ensino,area_conhecimento
0,ADMINISTRAÇÃO,GRADUAÇÃO,Ciências Sociais Aplicadas
1,ADMINISTRAÇÃO PÚBLICA,GRADUAÇÃO,Ciências Sociais Aplicadas
2,ANÁLISE E DESENVOLVIMENTO DE SISTEMAS,GRADUAÇÃO,Outra
3,ARQUITETURA E URBANISMO,GRADUAÇÃO,Engenharias
4,ARTES VISUAIS,GRADUAÇÃO,"Linguística, Letras e Artes"
5,BIBLIOTECONOMIA,GRADUAÇÃO,Ciências Sociais Aplicadas
6,BIOMEDICINA,GRADUAÇÃO,Ciências Biológicas
7,CIÊNCIA DA COMPUTAÇÃO,GRADUAÇÃO,Ciências Exatas e da Terra
8,CIÊNCIAS ATUARIAIS,GRADUAÇÃO,Ciências Exatas e da Terra
9,CIÊNCIAS BIOLÓGICAS,GRADUAÇÃO,Ciências Biológicas
